In [1]:
import igl
import math
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import os
root_folder = os.getcwd()

In [2]:
## Load a mesh in OFF format

#v, f = igl.read_triangle_mesh(os.path.join(root_folder, "Coarse_Mesh_Sphere.off"))
###Bending Energy calculation
def fun_EB(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

    minv = sp.sparse.diags(1 / m.diagonal())

    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    c=len(v)
    Kb=1 #bending_modulus
    Eb=[] # bending_energy
    for i in range (c):
        Eb.append((Kb/2)*((h_mean[i])**2)*(area_voronoi[i]))
        Eb_array = np.array(Eb)
    return Eb_array
#print(Eb)

In [3]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "CUBEMESH.off"))
b=fun_EB(v,f)

np.sum(b)

86.56830422613497

In [4]:
##Bending Force Calculation 
def fun_ForceDensity(v,f):

    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

    minv = sp.sparse.diags(1 / m.diagonal())

    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    Lap_H=minv*l*(h_mean/2)
    first_product=h_mean*(((h_mean/2)**2)-k)
    second_product=np.add(Lap_H,first_product)
    n = igl.per_vertex_normals(v, f)
    normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    Kb=1
    #Force_density=[] # Force_Density
    Force_Nodal=[]

    #for i in range (len(v)):
    Force_density=2*Kb*(normal_v*second_product[:,None])
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [5]:
#### Random Force Calculation
def fun_FR(v,f):
    gamma=100
    Kb_T=0.01
    del_T=0.01
    sigma=math.sqrt(2*gamma*Kb_T/del_T)
    gaussian_noise=np.random.normal(0, 1, 1)
    adjacent_vertices=igl.adjacency_list(f)
    FRij_nodes=[]
    for i in range(len(v)):
        for j in range(len(adjacent_vertices[i])):
            k=adjacent_vertices[i][j]
            FD_ij=0
            rij=v[i]-v[k]
            rij_norm = rij/np.linalg.norm(rij)
            gaussian_noise=np.random.normal(0, 1, 1)
            FR_ij=FD_ij+(sigma*(np.random.normal(0, 1, 1))*rij_norm)
        FRij_nodes.append(FR_ij)
    return FRij_nodes


In [6]:
###Calculation of damping" force
# Mesh in (v, f)
def fun_FD(v,f,velocity):
    gamma=100
    Kb_T=0.01
    del_T=0.01
    sigma=math.sqrt(2*gamma*Kb_T/del_T)
    adjacent_vertices=igl.adjacency_list(f)
    FDij_nodes=[]
    for i in range(len(v)):
        for j in range(len(adjacent_vertices[i])):
            FD_ij=0
            k=adjacent_vertices[i][j]
            vij=vel_init[i]-vel_init[k]
            rij=v[i]-v[k]
            #rij_distance=math.sqrt((rij[0][0]**2)+(rij[0][1]**2)+(rij[0][2]**2))
            rij_norm = rij/np.linalg.norm(rij)
            FD_ij=FD_ij+(-((gamma*np.dot(vij, rij)*rij_norm)))
        FDij_nodes.append(FD_ij)
    return FDij_nodes

In [7]:
vel_init=np.zeros((len(v),3));
FB=np.array(fun_ForceDensity(v,f)) ##bending_force
FR=np.array(fun_FR(v,f))#random_force
FD=np.array(fun_FD(v,f,vel_init))###dissipative_force

FB

array([[-1.42217807e+01, -2.84435614e+01, -1.42217807e+01],
       [ 6.90636815e+00,  6.90636815e+00, -0.00000000e+00],
       [ 9.98397993e+00, -0.00000000e+00,  9.98397993e+00],
       [-4.40256189e+01,  0.00000000e+00,  0.00000000e+00],
       [ 3.11817415e+01, -0.00000000e+00,  3.11817415e+01],
       [ 6.05364399e+01, -0.00000000e+00,  6.05364399e+01],
       [-5.61029451e+01,  0.00000000e+00,  0.00000000e+00],
       [-3.20182977e+01,  0.00000000e+00,  0.00000000e+00],
       [ 2.03735044e+01,  2.03735044e+01, -0.00000000e+00],
       [-6.68634612e+01,  0.00000000e+00,  0.00000000e+00],
       [-5.65315917e-14,  0.00000000e+00,  0.00000000e+00],
       [ 4.62745991e+01, -0.00000000e+00,  4.62745991e+01],
       [-6.01572236e-14,  0.00000000e+00,  0.00000000e+00],
       [-3.18310685e+01,  0.00000000e+00,  0.00000000e+00],
       [ 3.43201507e+01,  3.43201507e+01, -0.00000000e+00],
       [ 6.41405926e-14, -0.00000000e+00, -0.00000000e+00],
       [ 1.24810721e-14, -0.00000000e+00

In [14]:
def fun_totalpressure(FB,FR,FD):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = np.array(sp.sparse.diags(1 / m.diagonal()))

    area_voronoi=np.array(m.diagonal())
    vel_init=np.zeros((len(v),3))
    #EB=np.array(fun_EB(v,f))


    physicalpressure=FB/area_voronoi[:,None]
    numericalpressure=(FR/area_voronoi[:,None]+FD/area_voronoi[:,None])
    totalpressure=physicalpressure+numericalpressure
    return totalpressure

In [20]:
totalpressure=fun_totalpressure(FB,FR,FD)
totalpressure

array([[-1.26261867e+03, -1.37871299e+03, -6.89356495e+02],
       [-3.41778393e+02,  2.63823139e+02, -4.32572345e+02],
       [ 7.14271545e+02,  5.59678952e+02,  3.58112140e+02],
       [-1.78640105e+03, -1.84650689e+02,  2.46200967e+02],
       [ 2.70779786e+03,  4.05170047e+02,  1.49228869e+03],
       [ 2.78501423e+03, -2.60596113e+02,  3.39307169e+03],
       [-2.68090530e+03,  4.13783222e+02,  1.40686168e+03],
       [-1.26168676e+03,  0.00000000e+00, -3.84777859e+02],
       [ 4.47529255e+02,  8.39187806e+02, -3.91658551e+02],
       [-3.21501520e+03,  4.37702887e+02,  4.37702887e+02],
       [-2.31174162e-12,  6.44333616e+02,  2.89950104e+02],
       [ 2.15599543e+03, -9.99609927e+01,  2.23929630e+03],
       [-2.39737193e-12,  4.20428637e+00, -1.40142778e+01],
       [-1.57069518e+03,  9.45985278e+01, -6.62191057e+02],
       [ 1.55640858e+03,  1.64445438e+03, -8.39976691e+01],
       [ 3.54117853e-12, -2.78894349e+01, -1.12488062e+03],
       [ 7.76602036e-13,  2.28136482e+02

In [27]:
### Velocity Verlet Integration


dt=0.01
hdt=0.5*dt
iterations = 100
timestep = 0.005
T=1
vel_new_half=np.zeros((len(v),3))
pos_new_half=np.zeros((len(v),3))
vel_new=np.zeros((len(v),3))
pos_new=np.zeros((len(v),3))

for i in range(iterations):
    for j in range(len(v)):
        vel_new_half[j]=vel_init[j]+0.5*totalpressure[j]*hdt
        pos_new[j]=v[j]+vel_new_half[j]*dt
    
    FB_new=np.array(fun_ForceDensity(pos_new,f)) ##bending_force
    FR_new=np.array(fun_FR(pos_new,f))#random_force
    FD_new=np.array(fun_FD(pos_new,f,vel_new_half))
    totalpressure_new=fun_totalpressure(FB_new,FD_new,FR_new)
    
    for k in range(len(v)):
        vel_new[k]=vel_new_half[k]+0.5*totalpressure_new[k]*dt
        vel_init[k]=vel_new[k]
        totalpressure[k]=totalpressure_new[k]
    #otalpressure=totalpressure_new
              

In [28]:
pos_new[0]-v[0]

array([nan, nan, nan])

In [26]:
plot(pos_new,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(nan, nan,…

In [ ]:
np.savetxt('position_new.txt',pos_new, delimiter=',')

In [ ]:
pos_new[1]-v[1]

In [ ]:
plot(v,f)